In [1]:
from sentence_transformers import SentenceTransformer
import scipy
from sentence_transformers import SentenceTransformer, util
import json
import pandas as pd
import re
from tqdm import tqdm

/home/wuyuman/anaconda3/envs/chatglm/lib/python3.9/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [59]:
# model = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model = SentenceTransformer('distiluse-base-multilingual-cased-v1')

In [6]:

f = open("/home/wuyuman/Chatglm/data/CEs/elements.json",encoding='utf-8')
content = f.read()
elements = json.loads(content)
f = open("/home/wuyuman/Chatglm/data/articles/article_all.json",encoding='utf-8')
content = f.read()
articles = json.loads(content)
data=pd.read_csv('/home/wuyuman/Chatglm/test_data_all.csv')


In [46]:
all_data=pd.read_csv('/home/wuyuman/Chatglm/data/predict_crime/all_data.csv')
all_data=all_data[all_data['crime_num']==1]
all_data['accusation']=[re.sub('[][]', '', i)+'罪' for i in all_data['accusation']]
accusation_list=all_data['accusation'].drop_duplicates()

accusation_selected=[]
for accusation in accusation_list.tolist():
    if accusation in articles.keys() and accusation in [i+'罪' for i in elements.keys()]:
        accusation_selected.append(accusation)

all_data=all_data[all_data['accusation'].isin(accusation_selected)]
all_data=all_data[['fact', 'accusation', 'death_penalty', 'imprisonment','life_imprisonment', 'crime_num', 'predict_result']]
# all_data.to_csv('fact_selected.csv')

In [47]:
article_selected={}
for accusation in articles:
    if accusation in accusation_selected:
        article_selected[accusation]=articles[accusation]

# with open("/home/wuyuman/Chatglm/article_selected.json",'w',encoding='utf-8') as f:
#         json.dump(article_selected, f,ensure_ascii=False)

In [5]:
def retrivel(fact_data,model,article_selected,elements):
    data=fact_data.copy()
    article_retrievel_top10=[]
    article_retrievel_top5=[]
    article_retrievel_top3=[]
    crime_retrievel_top10=[]
    crime_retrievel_top5=[]
    crime_retrievel_top3=[]
    definition_and_article_retrievel_top10=[]
    definition_and_article_retrievel_top5=[]
    definition_and_article_retrievel_top3=[]

    article_content=[]
    crime_definition=[]
    definition_and_article=[]
    crimes=[]

    for accusation in article_selected:
        article_content.append(accusation+':'+article_selected[accusation]['内容'])
        crime_definition.append(elements[accusation[:-1]][0])
        definition_and_article.append(elements[accusation[:-1]][0]+article_selected[accusation]['内容'])
        crimes.append(accusation)
    article_representation=model.encode(article_content)
    crime_definition_representation=model.encode(crime_definition)
    definition_and_article_representation=model.encode(definition_and_article)

    for i in tqdm(range(len(data))):
        fact_representation=model.encode(all_data['fact'].iloc[i])

        correlation1={}
        correlation2={}
        correlation3={}
        for j in range(len(article_content)):
            correlation1[crimes[j]]= util.cos_sim(fact_representation,article_representation[j])
            correlation2[crimes[j]]= util.cos_sim(fact_representation,crime_definition_representation[j])
            correlation3[crimes[j]]= util.cos_sim(fact_representation,definition_and_article_representation[j])
        article_retrievel_top10.append(sorted(correlation1.items(),key = lambda correlation1:correlation1[1],reverse=True)[:10])
        article_retrievel_top5.append(sorted(correlation1.items(),key = lambda correlation1:correlation1[1],reverse=True)[:5])
        article_retrievel_top3.append(sorted(correlation1.items(),key = lambda correlation1:correlation1[1],reverse=True)[:3])
        crime_retrievel_top10.append(sorted(correlation2.items(),key = lambda correlation2:correlation2[1],reverse=True)[:10])
        crime_retrievel_top5.append(sorted(correlation2.items(),key = lambda correlation2:correlation2[1],reverse=True)[:5])
        crime_retrievel_top3.append(sorted(correlation2.items(),key = lambda correlation2:correlation2[1],reverse=True)[:3])
        definition_and_article_retrievel_top10.append(sorted(correlation3.items(),key = lambda correlation3:correlation3[1],reverse=True)[:10])
        definition_and_article_retrievel_top5.append(sorted(correlation3.items(),key = lambda correlation3:correlation3[1],reverse=True)[:5])
        definition_and_article_retrievel_top3.append(sorted(correlation3.items(),key = lambda correlation3:correlation3[1],reverse=True)[:3])

    data['top_5_article']=[[i[j][0] for j in range(len(i))] for i in article_retrievel_top5]
    data['top_3_article']=[[i[j][0] for j in range(len(i))] for i in article_retrievel_top3]
    data['top_10_article']=[[i[j][0] for j in range(len(i))] for i in article_retrievel_top10]
    data['top_5_crime']=[[i[j][0] for j in range(len(i))] for i in crime_retrievel_top5]
    data['top_3_crime']=[[i[j][0] for j in range(len(i))] for i in crime_retrievel_top3]
    data['top_10_crime']=[[i[j][0] for j in range(len(i))] for i in crime_retrievel_top10]
    data['top_5_definition_and_article']=[[i[j][0] for j in range(len(i))] for i in definition_and_article_retrievel_top5]
    data['top_3_definition_and_article']=[[i[j][0] for j in range(len(i))] for i in definition_and_article_retrievel_top3]
    data['top_10_definition_and_article']=[[i[j][0] for j in range(len(i))] for i in definition_and_article_retrievel_top10]
    return data
            
    

In [6]:
model1 = SentenceTransformer('distiluse-base-multilingual-cased-v1')
model2 = SentenceTransformer('paraphrase-multilingual-MiniLM-L12-v2')
model3=SentenceTransformer('/home/wuyuman/hfl/chinese-roberta-wwm-ext')

No sentence-transformers model found with name /home/wuyuman/hfl/chinese-roberta-wwm-ext. Creating a new one with MEAN pooling.
Some weights of the model checkpoint at /home/wuyuman/hfl/chinese-roberta-wwm-ext were not used when initializing BertModel: ['cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


In [7]:
data1=retrivel(fact_data=all_data,model=model1,article_selected=article_selected,elements=elements)
data2=retrivel(fact_data=all_data,model=model2,article_selected=article_selected,elements=elements)
data3=retrivel(fact_data=all_data,model=model3,article_selected=article_selected,elements=elements)

100%|██████████| 13994/13994 [07:04<00:00, 32.99it/s]


In [2]:
data1=pd.read_csv('/home/wuyuman/Chatglm/data/retrivel_data/sbert_retrivel1.csv')

In [4]:
data1[['accusation','top_5_article','top_5_crime','top_5_definition_and_article']].iloc[3000]

accusation                                                                  交通肇事罪
top_5_article                   ['交通肇事罪', '危险驾驶罪', '聚众扰乱公共场所秩序、交通秩序罪', '重大劳动安全...
top_5_crime                     ['交通肇事罪', '危险驾驶罪', '聚众扰乱公共场所秩序、交通秩序罪', '重大责任事故...
top_5_definition_and_article    ['交通肇事罪', '危险驾驶罪', '重大责任事故罪', '重大劳动安全事故罪', '聚众...
Name: 3000, dtype: object

In [9]:
data2[['accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,accusation,top_5_article,top_5_crime,top_5_definition_and_article
0,盗窃罪,"[故意毁坏财物罪, 抢夺罪, 持有、使用假币罪, 出售、购买、运输假币罪, 伪造、变造金融票证罪]","[抢夺罪, 集资诈骗罪, 单位行贿罪, 诬告陷害罪, 合同诈骗罪]","[抢夺罪, 票据诈骗罪, 集资诈骗罪, 持有、使用假币罪, 出售、购买、运输假币罪]"
1,盗窃罪,"[伪证罪, 诬告陷害罪, 伪造、变造金融票证罪, 非法处置查封、扣押、冻结的财产罪, 票据诈骗罪]","[集资诈骗罪, 票据诈骗罪, 合同诈骗罪, 诬告陷害罪, 职务侵占罪]","[诬告陷害罪, 伪证罪, 票据诈骗罪, 出售、购买、运输假币罪, 持有、使用假币罪]"
2,盗窃罪,"[诬告陷害罪, 伪证罪, 非法侵入住宅罪, 编造、故意传播虚假恐怖信息罪, 非法处置查封、扣...","[诬告陷害罪, 集资诈骗罪, 票据诈骗罪, 保险诈骗罪, 抢夺罪]","[诬告陷害罪, 伪证罪, 出售、购买、运输假币罪, 抢夺罪, 持有、使用假币罪]"
3,盗窃罪,"[非法处置查封、扣押、冻结的财产罪, 伪造公司、企业、事业单位、人民团体印章罪, 出售、购买...","[集资诈骗罪, 票据诈骗罪, 销售假冒注册商标的商品罪, 抢夺罪, 保险诈骗罪]","[抢夺罪, 出售、购买、运输假币罪, 盗窃罪, 掩饰、隐瞒犯罪所得、犯罪所得收益罪, 诬告陷害罪]"
4,盗窃罪,"[非法处置查封、扣押、冻结的财产罪, 出售、购买、运输假币罪, 持有、使用假币罪, 非法出售...","[集资诈骗罪, 合同诈骗罪, 保险诈骗罪, 票据诈骗罪, 抢夺罪]","[出售、购买、运输假币罪, 抢夺罪, 持有、使用假币罪, 非法出售发票罪, 非法处置查封、扣..."
...,...,...,...,...
17123,强迫交易罪,"[非法处置查封、扣押、冻结的财产罪, 非法转让、倒卖土地使用权罪, 销售假冒注册商标的商品罪...","[销售假冒注册商标的商品罪, 集资诈骗罪, 合同诈骗罪, 票据诈骗罪, 保险诈骗罪]","[非法转让、倒卖土地使用权罪, 非法占用农用地罪, 非法处置查封、扣押、冻结的财产罪, 编造..."
17127,妨害公务罪,"[诬告陷害罪, 帮助犯罪分子逃避处罚罪, 伪证罪, 招摇撞骗罪, 编造、故意传播虚假恐怖信息罪]","[职务侵占罪, 冒充军人招摇撞骗罪, 诬告陷害罪, 票据诈骗罪, 对非国家工作人员行贿罪]","[诬告陷害罪, 帮助犯罪分子逃避处罚罪, 伪证罪, 拒不执行判决、裁定罪, 玩忽职守罪]"
17128,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"[非法出售发票罪, 生产、销售伪劣产品罪, 出售、购买、运输假币罪, 虚开增值税专用发票、用...","[集资诈骗罪, 保险诈骗罪, 销售假冒注册商标的商品罪, 票据诈骗罪, 单位行贿罪]","[非法出售发票罪, 出售、购买、运输假币罪, 非法收购、运输盗伐、滥伐的林木罪, 保险诈骗罪..."
17129,窝藏、包庇罪,"[过失致人重伤罪, 拒不执行判决、裁定罪, 拐骗儿童罪, 招摇撞骗罪, 非法处置查封、扣押、...","[职务侵占罪, 票据诈骗罪, 集资诈骗罪, 单位行贿罪, 合同诈骗罪]","[故意杀人罪, 拒不执行判决、裁定罪, 故意伤害罪, 过失致人重伤罪, 拐骗儿童罪]"


In [10]:
data3[['accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,accusation,top_5_article,top_5_crime,top_5_definition_and_article
0,盗窃罪,"[对非国家工作人员行贿罪, 集资诈骗罪, 非法吸收公众存款罪, 绑架罪, 单位行贿罪]","[故意杀人罪, 诬告陷害罪, 故意毁坏财物罪, 侵犯著作权罪, 交通肇事罪]","[故意杀人罪, 对非国家工作人员行贿罪, 集资诈骗罪, 诬告陷害罪, 持有伪造的发票罪]"
1,盗窃罪,"[伪证罪, 非法拘禁罪, 侵犯著作权罪, 拒不支付劳动报酬罪, 票据诈骗罪]","[伪证罪, 掩饰、隐瞒犯罪所得、犯罪所得收益罪, 帮助犯罪分子逃避处罚罪, 违法发放贷款罪,...","[伪证罪, 非法拘禁罪, 侵犯著作权罪, 拒不支付劳动报酬罪, 伪造、变造金融票证罪]"
2,盗窃罪,"[侵犯著作权罪, 非法拘禁罪, 危险驾驶罪, 票据诈骗罪, 抢劫罪]","[掩饰、隐瞒犯罪所得、犯罪所得收益罪, 帮助犯罪分子逃避处罚罪, 故意杀人罪, 非法处置查封...","[非法侵入住宅罪, 侵犯著作权罪, 抢劫罪, 危险驾驶罪, 窝藏、包庇罪]"
3,盗窃罪,"[侵犯著作权罪, 信用卡诈骗罪, 非法拘禁罪, 票据诈骗罪, 合同诈骗罪]","[掩饰、隐瞒犯罪所得、犯罪所得收益罪, 帮助犯罪分子逃避处罚罪, 聚众扰乱公共场所秩序、交通...","[侵犯著作权罪, 抢劫罪, 危险驾驶罪, 信用卡诈骗罪, 票据诈骗罪]"
4,盗窃罪,"[侵犯著作权罪, 走私、贩卖、运输、制造毒品罪, 票据诈骗罪, 污染环境罪, 非法拘禁罪]","[组织、领导传销活动罪, 违法发放贷款罪, 掩饰、隐瞒犯罪所得、犯罪所得收益罪, 挪用资金罪...","[侵犯著作权罪, 走私、贩卖、运输、制造毒品罪, 票据诈骗罪, 挪用资金罪, 非法拘禁罪]"
...,...,...,...,...
17123,强迫交易罪,"[侵犯著作权罪, 挪用资金罪, 走私、贩卖、运输、制造毒品罪, 污染环境罪, 拒不支付劳动报酬罪]","[违法发放贷款罪, 组织、领导传销活动罪, 滥伐林木罪, 非法采矿罪, 挪用资金罪]","[侵犯著作权罪, 违法发放贷款罪, 挪用资金罪, 拒不支付劳动报酬罪, 走私、贩卖、运输、制..."
17127,妨害公务罪,"[非法拘禁罪, 伪证罪, 妨害公务罪, 侵犯著作权罪, 编造、故意传播虚假恐怖信息罪]","[伪证罪, 妨害公务罪, 聚众扰乱公共场所秩序、交通秩序罪, 帮助犯罪分子逃避处罚罪, 违法...","[非法侵入住宅罪, 伪证罪, 编造、故意传播虚假恐怖信息罪, 妨害公务罪, 非法拘禁罪]"
17128,掩饰、隐瞒犯罪所得、犯罪所得收益罪,"[走私、贩卖、运输、制造毒品罪, 侵犯著作权罪, 非法拘禁罪, 票据诈骗罪, 拐卖妇女、儿童罪]","[掩饰、隐瞒犯罪所得、犯罪所得收益罪, 组织、领导传销活动罪, 违法发放贷款罪, 挪用资金罪...","[走私、贩卖、运输、制造毒品罪, 侵犯著作权罪, 非法拘禁罪, 拐卖妇女、儿童罪, 掩饰、隐..."
17129,窝藏、包庇罪,"[非法拘禁罪, 侵犯著作权罪, 污染环境罪, 走私、贩卖、运输、制造毒品罪, 拒不支付劳动报酬罪]","[伪证罪, 聚众斗殴罪, 组织、领导传销活动罪, 滥伐林木罪, 违法发放贷款罪]","[非法拘禁罪, 侵犯著作权罪, 聚众斗殴罪, 编造、故意传播虚假恐怖信息罪, 走私、贩卖、运..."


In [16]:
data1.to_csv('sbert_retrivel1.csv')
data2.to_csv('sbert_retrivel2.csv')
data3.to_csv('sbert_retrivel3.csv')

In [2]:
def retrievel_acc(data,column1,column2,column3):
    acc1=0
    acc2=0
    acc3=0
    for i in range(len(data)):
        if data['accusation'].iloc[i] in data[column1].iloc[i]:
            acc1+=1
        if data['accusation'].iloc[i] in data[column2].iloc[i]:
            acc2+=1
        if data['accusation'].iloc[i] in data[column3].iloc[i]:
            acc3+=1
    return column1+':'+str(acc1),column2+':'+str(acc2),column3+':'+str(acc3)

In [3]:
def each_acc(data,column):
    acc_dict={}
    num_dict={}
    for i in data['accusation'].value_counts().index:
        data_new=data[data['accusation']==i]
        sum=0
        for j in range(len(data_new)):
            if data_new['accusation'].iloc[j] in data_new[column].iloc[j]:
                sum+=1
        acc_dict[i]=sum/len(data_new)
    return acc_dict

In [3]:
data1=pd.read_csv('/home/wuyuman/Chatglm/data/retrivel_data/sbert_retrivel1.csv')
data2=pd.read_csv('/home/wuyuman/Chatglm/data/retrivel_data/sbert_retrivel2.csv')
data3=pd.read_csv('/home/wuyuman/Chatglm/data/retrivel_data/sbert_retrivel3.csv')

In [20]:
for retrie_article in data1['top_5_article']:
    knowledge='法条规定如下。'
    for i in eval(retrie_article):
        knowledge+=(i+'：'+articles[i]['内容'])
    print(knowledge)
    break

法条规定如下。盗窃罪：盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金;数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金;数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。非法处置查封、扣押、冻结的财产罪：隐藏、转移、变卖、故意毁损已被司法机关查封、扣押、冻结的财产，情节严重的，处三年以下有期徒刑、拘役或者罚金。抢夺罪：抢夺公私财物，数额较大的，或者多次抢夺的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金;数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金;数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。携带凶器抢夺的，依照本法第二百六十三条的规定定罪处罚。职务侵占罪：公司、企业或者其他单位的工作人员，利用职务上的便利，将本单位财物非法占为己有，数额较大的，处三年以下有期徒刑或者拘役，并处罚金；数额巨大的，处三年以上十年以下有期徒刑，并处罚金；数额特别巨大的，处十年以上有期徒刑或者无期徒刑，并处罚金。国有公司、企业或者其他国有单位中从事公务的人员和国有公司、企业或者其他国有单位委派到非国有公司、企业以及其他单位从事公务的人员有前款行为的，依照本法第三百八十二条、第三百八十三条的规定定罪处罚。对非国家工作人员行贿罪：为谋取不正当利益，给予公司、企业或者其他单位的工作人员以财物，数额较大的，处三年以下有期徒刑或者拘役，并处罚金;数额巨大的，处三年以上十年以下有期徒刑，并处罚金。为谋取不正当商业利益，给予外国公职人员或者国际公共组织官员以财物的，依照前款的规定处罚。单位犯前两款罪的，对单位判处罚金，并对其直接负责的主管人员和其他直接责任人员，依照第一款的规定处罚。行贿人在被追诉前主动交待行贿行为的，可以减轻处罚或者免除处罚。


In [21]:
data1['top_5_article'][:10]

0    ['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '职务侵占罪', '对...
1          ['交通肇事罪', '诬告陷害罪', '票据诈骗罪', '寻衅滋事罪', '诈骗罪']
2              ['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢劫罪', '伪证罪']
3    ['伪证罪', '玩忽职守罪', '诬告陷害罪', '招摇撞骗罪', '非法处置查封、扣押、...
4    ['非法吸收公众存款罪', '出售、购买、运输假币罪', '非法处置查封、扣押、冻结的财产罪...
5    ['诬告陷害罪', '行贿罪', '寻衅滋事罪', '单位行贿罪', '对非国家工作人员行贿罪']
6    ['交通肇事罪', '非法拘禁罪', '非法处置查封、扣押、冻结的财产罪', '招摇撞骗罪'...
7        ['敲诈勒索罪', '寻衅滋事罪', '诬告陷害罪', '单位行贿罪', '滥用职权罪']
8    ['非法吸收公众存款罪', '行贿罪', '盗窃罪', '非法处置查封、扣押、冻结的财产罪'...
9              ['盗窃罪', '伪证罪', '诈骗罪', '诬告陷害罪', '敲诈勒索罪']
Name: top_5_article, dtype: object

In [5]:
retrievel_acc(data=data1,column1='top_5_article',column2='top_5_crime',column3='top_5_definition_and_article')

('top_5_article:7375', 'top_5_crime:7190', 'top_5_definition_and_article:7225')

In [25]:
data1[data1['accusation']=='盗窃罪'][['accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

,accusation,top_5_article,top_5_crime,top_5_definition_and_article
0,盗窃罪,"['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '职务侵占罪', '对...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '抢夺罪', '侵犯著作权罪', '...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '职务侵占罪', '抢夺罪', '非..."
1,盗窃罪,"['交通肇事罪', '诬告陷害罪', '票据诈骗罪', '寻衅滋事罪', '诈骗罪']","['交通肇事罪', '寻衅滋事罪', '诬告陷害罪', '非法处置查封、扣押、冻结的财产罪'...","['诬告陷害罪', '交通肇事罪', '票据诈骗罪', '单位行贿罪', '寻衅滋事罪']"
2,盗窃罪,"['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢劫罪', '伪证罪']","['交通肇事罪', '盗窃罪', '抢夺罪', '抢劫罪', '寻衅滋事罪']","['交通肇事罪', '盗窃罪', '招摇撞骗罪', '抢夺罪', '冒充军人招摇撞骗罪']"
3,盗窃罪,"['伪证罪', '玩忽职守罪', '诬告陷害罪', '招摇撞骗罪', '非法处置查封、扣押、...","['非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '寻衅滋事罪', '妨害公务罪...","['冒充军人招摇撞骗罪', '非法处置查封、扣押、冻结的财产罪', '持有、使用假币罪', ..."
4,盗窃罪,"['非法吸收公众存款罪', '出售、购买、运输假币罪', '非法处置查封、扣押、冻结的财产罪...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '抢夺罪', '妨害公务...","['非法处置查封、扣押、冻结的财产罪', '非法吸收公众存款罪', '出售、购买、运输假币罪..."
...,...,...,...,...
2178,盗窃罪,"['盗窃罪', '寻衅滋事罪', '抢劫罪', '敲诈勒索罪', '出售、购买、运输假币罪']","['盗窃罪', '抢劫罪', '非法处置查封、扣押、冻结的财产罪', '非法收购、运输盗伐、...","['盗窃罪', '寻衅滋事罪', '敲诈勒索罪', '抢劫罪', '非法处置查封、扣押、冻结..."
2179,盗窃罪,"['信用卡诈骗罪', '集资诈骗罪', '票据诈骗罪', '持有、使用假币罪', '诈骗罪']","['骗取贷款、票据承兑、金融票证罪', '持有、使用假币罪', '信用卡诈骗罪', '诈骗罪...","['信用卡诈骗罪', '集资诈骗罪', '票据诈骗罪', '持有、使用假币罪', '贷款诈骗罪']"
2180,盗窃罪,"['招摇撞骗罪', '滥用职权罪', '伪证罪', '诬告陷害罪', '非法拘禁罪']","['非法处置查封、扣押、冻结的财产罪', '行贿罪', '招摇撞骗罪', '伪证罪', '交...","['招摇撞骗罪', '冒充军人招摇撞骗罪', '单位行贿罪', '诬告陷害罪', '非法拘禁罪']"
2181,盗窃罪,"['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '诈骗罪', '寻衅滋事罪', '伪...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '抢夺罪', '...","['盗窃罪', '非法处置查封、扣押、冻结的财产罪', '侵犯著作权罪', '票据诈骗罪',..."


In [35]:
f_crime=[]
for i in range(len(data1[data1['accusation']=='盗窃罪'])):
    if data1['accusation'].iloc[i] in data1['top_5_article'].iloc[i]:
        pass
    else:
        f_crime.extend(eval(data1['top_3_article'].iloc[i]))

In [67]:
import jieba

'河南省 舞钢市 人民检察院 指控 2015 年 6 月 1 日 20 时许 被告人 李 1 舞钢市 朱兰 干休 路 南段 兰兴 宾馆 门口 白 某某 停放 此处 立马 牌 电动车 盗走 当庭 出示 被告人 供述 被害人 陈述 证人 证言 鉴定 意见 相关 物证 书证 证据 公诉 机关 依据 上述事实 证据 认为 被告人 李 1 行为 已 构成 罪 请求 中华人民共和国 刑法 规定 追究其 刑事责任'

In [66]:
stopwords = open('/home/wuyuman/Chatglm/hit_stopwords.txt').read().split('\n')
def clean_token(sentence_tokens):
    tokens = []
    for word in sentence_tokens:
      if word in stopwords:
        continue
      else:
        tokens.append(word)
    return tokens

In [ ]:
' '.join(clean_token(jieba.lcut(articles['寻衅滋事罪']['内容'])))
' '.join(clean_token(jieba.lcut(articles['寻衅滋事罪']['内容'])))
' '.join(clean_token(jieba.lcut(articles['寻衅滋事罪']['内容'])))
' '.join(clean_token(jieba.lcut(articles['寻衅滋事罪']['内容'])))
' '.join(clean_token(jieba.lcut(articles['寻衅滋事罪']['内容'])))

In [72]:
fact=model.encode(' '.join(jieba.lcut(data1.loc[2180,'fact'])))

art1=model.encode(' '.join(jieba.lcut(articles['寻衅滋事罪']['内容'])))
art2=model.encode(' '.join(jieba.lcut(articles['招摇撞骗罪']['内容'])))
art3=model.encode(' '.join(jieba.lcut(articles['滥用职权罪']['内容'])))
art4=model.encode(' '.join(jieba.lcut(articles['盗窃罪']['内容'])))
art5=model.encode(' '.join(jieba.lcut(articles['伪证罪']['内容'])))
r1=util.cos_sim(fact,art1)
r2=util.cos_sim(fact,art2)
r3=util.cos_sim(fact,art3)
r4=util.cos_sim(fact,art4)
r5=util.cos_sim(fact,art5)
print(r1,r2,r3,r4,r5)

tensor([[0.3137]]) tensor([[0.3635]]) tensor([[0.4065]]) tensor([[0.3298]]) tensor([[0.3870]])


In [50]:
articles['寻衅滋事罪']['内容']

'有下列寻衅滋事行为之一，破坏社会秩序的，处五年以下有期徒刑、拘役或者管制：(一)随意殴打他人，情节恶劣的;(二)追逐、拦截、辱骂、恐吓他人，情节恶劣的;(三)强拿硬要或者任意损毁、占用公私财物，情节严重的;(四)在公共场所起哄闹事，造成公共场所秩序严重混乱的。纠集他人多次实施前款行为，严重破坏社会秩序的，处五年以上十年以下有期徒刑，可以并处罚金。'

In [52]:
articles['招摇撞骗罪']['内容']

'冒充国家机关工作人员招摇撞骗的，处三年以下有期徒刑、拘役、管制或者剥夺政治权利;情节严重的，处三年以上十年以下有期徒刑。冒充人民警察招摇撞骗的，依照前款的规定从重处罚。'

In [57]:
articles['滥用职权罪']['内容']

'国家机关工作人员滥用职权或者玩忽职守，致使公共财产、国家和人民利益遭受重大损失的，处三年以下有期徒刑或者拘役;情节特别严重的，处三年以上七年以下有期徒刑。本法另有规定的，依照规定。国家机关工作人员徇私舞弊，犯前款罪的，处五年以下有期徒刑或者拘役;情节特别严重的，处五年以上十年以下有期徒刑。本法另有规定的，依照规定。'

In [58]:
articles['伪证罪']['内容']

'在刑事诉讼中，证人、鉴定人、记录人、翻译人对与案件有重要关系的情节，故意作虚假证明、鉴定、记录、翻译，意图陷害他人或者隐匿罪证的，处三年以下有期徒刑或者拘役;情节严重的，处三年以上七年以下有期徒刑。'

In [62]:
articles['盗窃罪']['内容']

'盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金;数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金;数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。'

In [55]:
data1.loc[2180,'fact']

'河南省舞钢市人民检察院指控，2015年6月1日20时许，被告人李1某在舞钢市朱兰干休路南段兰兴宾馆门口将白某某停放在此处的立马牌电动车盗走。并当庭出示了被告人供述、被害人陈述、证人证言、鉴定意见、相关物证、书证等证据。公诉机关依据上述事实、证据，认为被告人李1某行为已构成××罪，请求依照《中华人民共和国刑法》××之规定追究其刑事责任。'

In [73]:
data1.loc[2180,'top_3_article']

"['招摇撞骗罪', '滥用职权罪', '伪证罪']"

In [51]:
articles['盗窃罪']['内容']

'盗窃公私财物，数额较大的，或者多次盗窃、入户盗窃、携带凶器盗窃、扒窃的，处三年以下有期徒刑、拘役或者管制，并处或者单处罚金;数额巨大或者有其他严重情节的，处三年以上十年以下有期徒刑，并处罚金;数额特别巨大或者有其他特别严重情节的，处十年以上有期徒刑或者无期徒刑，并处罚金或者没收财产。'

In [53]:
elements['寻衅滋事'][0]

'寻衅滋事罪是指在公共场所无事生非，起哄闹事，殴打伤害无辜，肆意挑衅，横行霸道，破坏公共秩序的行为。'

In [24]:
each_acc(data=data1,column='top_5_crime')

{'盗窃罪': 0.47924528301886793,
 '走私、贩卖、运输、制造毒品罪': 0.9252577319587629,
 '故意伤害罪': 0.3157051282051282,
 '诈骗罪': 0.4208955223880597,
 '寻衅滋事罪': 0.7922077922077922,
 '抢劫罪': 0.2935153583617747,
 '危险驾驶罪': 0.985663082437276,
 '合同诈骗罪': 0.007874015748031496,
 '故意杀人罪': 0.4885844748858447,
 '非法占用农用地罪': 0.9607843137254902,
 '信用卡诈骗罪': 0.896551724137931,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.7178217821782178,
 '强奸罪': 0.26732673267326734,
 '滥伐林木罪': 0.9698492462311558,
 '非法持有、私藏枪支、弹药罪': 0.9698492462311558,
 '交通肇事罪': 0.9949238578680203,
 '非法经营罪': 0.10309278350515463,
 '生产、销售有毒、有害食品罪': 0.6839378238341969,
 '过失致人死亡罪': 0.20207253886010362,
 '生产、销售不符合安全标准的食品罪': 0.5625,
 '污染环境罪': 0.4166666666666667,
 '非法行医罪': 0.984375,
 '非法种植毒品原植物罪': 0.9010416666666666,
 '假冒注册商标罪': 0.612565445026178,
 '非法吸收公众存款罪': 0.3612565445026178,
 '销售假冒注册商标的商品罪': 0.4842105263157895,
 '拒不支付劳动报酬罪': 0.7473684210526316,
 '盗伐林木罪': 0.9368421052631579,
 '拒不执行判决、裁定罪': 0.6613756613756614,
 '故意毁坏财物罪': 0.08465608465608465,
 '职务侵占罪': 0.005291005291005291,
 '

In [17]:
each_acc(data=data1,column='top_5_article')

{'盗窃罪': 0.35471698113207545,
 '走私、贩卖、运输、制造毒品罪': 0.9574742268041238,
 '故意伤害罪': 0.5673076923076923,
 '诈骗罪': 0.6447761194029851,
 '寻衅滋事罪': 0.6396103896103896,
 '抢劫罪': 0.21160409556313994,
 '危险驾驶罪': 0.985663082437276,
 '合同诈骗罪': 0.18503937007874016,
 '故意杀人罪': 0.2420091324200913,
 '非法占用农用地罪': 0.8578431372549019,
 '信用卡诈骗罪': 0.9507389162561576,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.5643564356435643,
 '强奸罪': 0.26732673267326734,
 '滥伐林木罪': 0.9547738693467337,
 '非法持有、私藏枪支、弹药罪': 0.9798994974874372,
 '交通肇事罪': 0.9949238578680203,
 '非法经营罪': 0.015463917525773196,
 '生产、销售有毒、有害食品罪': 0.5595854922279793,
 '过失致人死亡罪': 0.15025906735751296,
 '生产、销售不符合安全标准的食品罪': 0.6197916666666666,
 '污染环境罪': 0.6614583333333334,
 '非法行医罪': 0.984375,
 '非法种植毒品原植物罪': 0.9010416666666666,
 '假冒注册商标罪': 0.5340314136125655,
 '非法吸收公众存款罪': 0.33507853403141363,
 '销售假冒注册商标的商品罪': 0.6684210526315789,
 '拒不支付劳动报酬罪': 0.7578947368421053,
 '盗伐林木罪': 0.9052631578947369,
 '拒不执行判决、裁定罪': 0.8042328042328042,
 '故意毁坏财物罪': 0.15873015873015872,
 '职务侵占罪': 0.338624

In [18]:
retrievel_acc(data=data2,column1='top_5_article',column2='top_5_crime',column3='top_5_definition_and_article')

('top_5_article:6483', 'top_5_crime:4726', 'top_5_definition_and_article:7027')

In [20]:
each_acc(data=data2,column='top_5_article')

{'盗窃罪': 0.08427672955974842,
 '走私、贩卖、运输、制造毒品罪': 0.8337628865979382,
 '故意伤害罪': 0.4310897435897436,
 '诈骗罪': 0.2955223880597015,
 '寻衅滋事罪': 0.1525974025974026,
 '抢劫罪': 0.0034129692832764505,
 '危险驾驶罪': 0.8960573476702509,
 '合同诈骗罪': 0.047244094488188976,
 '故意杀人罪': 0.1004566210045662,
 '非法占用农用地罪': 0.9411764705882353,
 '信用卡诈骗罪': 0.8866995073891626,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.801980198019802,
 '强奸罪': 0.16336633663366337,
 '滥伐林木罪': 0.7185929648241206,
 '非法持有、私藏枪支、弹药罪': 0.964824120603015,
 '交通肇事罪': 0.9949238578680203,
 '非法经营罪': 0.20618556701030927,
 '生产、销售有毒、有害食品罪': 0.43005181347150256,
 '过失致人死亡罪': 0.11398963730569948,
 '生产、销售不符合安全标准的食品罪': 0.4375,
 '污染环境罪': 0.828125,
 '非法行医罪': 0.9479166666666666,
 '非法种植毒品原植物罪': 0.953125,
 '假冒注册商标罪': 0.4031413612565445,
 '非法吸收公众存款罪': 0.42408376963350786,
 '销售假冒注册商标的商品罪': 0.7947368421052632,
 '拒不支付劳动报酬罪': 0.8789473684210526,
 '盗伐林木罪': 0.7105263157894737,
 '拒不执行判决、裁定罪': 0.8412698412698413,
 '故意毁坏财物罪': 0.36507936507936506,
 '职务侵占罪': 0.07407407407407407,
 '聚众斗殴罪

In [19]:
retrievel_acc(data=data3,column1='top_5_article',column2='top_5_crime',column3='top_5_definition_and_article')

('top_5_article:3883', 'top_5_crime:4044', 'top_5_definition_and_article:4751')

In [21]:
each_acc(data=data3,column='top_5_article')

{'盗窃罪': 0.0,
 '走私、贩卖、运输、制造毒品罪': 0.9858247422680413,
 '故意伤害罪': 0.0,
 '诈骗罪': 0.982089552238806,
 '寻衅滋事罪': 0.09415584415584416,
 '抢劫罪': 0.020477815699658702,
 '危险驾驶罪': 0.992831541218638,
 '合同诈骗罪': 0.7598425196850394,
 '故意杀人罪': 0.0,
 '非法占用农用地罪': 0.23529411764705882,
 '信用卡诈骗罪': 0.8719211822660099,
 '虚开增值税专用发票、用于骗取出口退税、抵扣税款发票罪': 0.9702970297029703,
 '强奸罪': 0.1188118811881188,
 '滥伐林木罪': 0.0,
 '非法持有、私藏枪支、弹药罪': 0.7085427135678392,
 '交通肇事罪': 0.07614213197969544,
 '非法经营罪': 0.07731958762886598,
 '生产、销售有毒、有害食品罪': 0.015544041450777202,
 '过失致人死亡罪': 0.0,
 '生产、销售不符合安全标准的食品罪': 0.0,
 '污染环境罪': 0.9895833333333334,
 '非法行医罪': 0.0,
 '非法种植毒品原植物罪': 0.9479166666666666,
 '假冒注册商标罪': 0.0,
 '非法吸收公众存款罪': 0.010471204188481676,
 '销售假冒注册商标的商品罪': 0.0,
 '拒不支付劳动报酬罪': 0.9947368421052631,
 '盗伐林木罪': 0.0,
 '拒不执行判决、裁定罪': 0.026455026455026454,
 '故意毁坏财物罪': 0.0,
 '职务侵占罪': 0.0,
 '聚众斗殴罪': 0.14893617021276595,
 '行贿罪': 0.6203208556149733,
 '敲诈勒索罪': 0.0,
 '窝藏、包庇罪': 0.14516129032258066,
 '放火罪': 0.0,
 '妨害公务罪': 0.3172043010752688,
 '非国家工作

In [92]:
crime_selected=['故意伤害', '诈骗', '寻衅滋事', '抢劫', '危险驾驶', '绑架','受贿','遗弃', '合同诈骗', '赌博','故意杀人', '侵犯著作权', '信用卡诈骗', '强奸','交通肇事','非法经营','过失致人死亡']

In [93]:
selected_data=all_data[all_data['accusation'].isin([i+'罪' for i in crime_selected])][['accusation','top_5_article','top_5_crime','top_5_definition_and_article']]

In [94]:
selected_data

,accusation,top_5_article,top_5_crime,top_5_definition_and_article
355,故意伤害罪,"[聚众斗殴罪, 故意伤害罪, 寻衅滋事罪, 诬告陷害罪, 过失致人重伤罪]","[聚众斗殴罪, 寻衅滋事罪, 过失致人重伤罪, 诬告陷害罪, 敲诈勒索罪]","[聚众斗殴罪, 寻衅滋事罪, 诬告陷害罪, 单位行贿罪, 故意伤害罪]"
361,故意伤害罪,"[寻衅滋事罪, 故意伤害罪, 滥用职权罪, 单位行贿罪, 交通肇事罪]","[寻衅滋事罪, 滥用职权罪, 敲诈勒索罪, 交通肇事罪, 过失致人重伤罪]","[单位行贿罪, 寻衅滋事罪, 故意伤害罪, 敲诈勒索罪, 诬告陷害罪]"
363,故意伤害罪,"[寻衅滋事罪, 非法行医罪, 诬告陷害罪, 招摇撞骗罪, 滥用职权罪]","[寻衅滋事罪, 非法处置查封、扣押、冻结的财产罪, 滥用职权罪, 交通肇事罪, 非法行医罪]","[寻衅滋事罪, 招摇撞骗罪, 单位行贿罪, 诬告陷害罪, 非法行医罪]"
364,故意伤害罪,"[非法拘禁罪, 寻衅滋事罪, 非法处置查封、扣押、冻结的财产罪, 诬告陷害罪, 帮助犯罪分子...","[非法处置查封、扣押、冻结的财产罪, 寻衅滋事罪, 故意杀人罪, 帮助犯罪分子逃避处罚罪, ...","[非法处置查封、扣押、冻结的财产罪, 寻衅滋事罪, 冒充军人招摇撞骗罪, 诬告陷害罪, 非法..."
368,故意伤害罪,"[寻衅滋事罪, 交通肇事罪, 招摇撞骗罪, 聚众斗殴罪, 危险驾驶罪]","[寻衅滋事罪, 交通肇事罪, 聚众斗殴罪, 聚众扰乱公共场所秩序、交通秩序罪, 非法转让、倒...","[寻衅滋事罪, 交通肇事罪, 聚众斗殴罪, 冒充军人招摇撞骗罪, 招摇撞骗罪]"
...,...,...,...,...
16336,遗弃罪,"[寻衅滋事罪, 窝藏、包庇罪, 非法侵入住宅罪, 拒不执行判决、裁定罪, 玩忽职守罪]","[寻衅滋事罪, 窝藏、包庇罪, 非法进行节育手术罪, 非法处置查封、扣押、冻结的财产罪, 非...","[寻衅滋事罪, 非法侵入住宅罪, 窝藏、包庇罪, 拒不执行判决、裁定罪, 单位行贿罪]"
16368,遗弃罪,"[诬告陷害罪, 拒不执行判决、裁定罪, 伪证罪, 滥用职权罪, 非法处置查封、扣押、冻结的财产罪]","[拒不执行判决、裁定罪, 诬告陷害罪, 非法处置查封、扣押、冻结的财产罪, 伪证罪, 滥用职权罪]","[诬告陷害罪, 拒不执行判决、裁定罪, 伪证罪, 非法处置查封、扣押、冻结的财产罪, 滥用职权罪]"
16382,绑架罪,"[非法持有毒品罪, 聚众扰乱社会秩序罪, 聚众扰乱公共场所秩序、交通秩序罪, 走私、贩卖、运...","[妨害公务罪, 故意杀人罪, 聚众扰乱公共场所秩序、交通秩序罪, 非法种植毒品原植物罪, 走...","[走私、贩卖、运输、制造毒品罪, 故意杀人罪, 非法持有毒品罪, 非法种植毒品原植物罪, 非..."
16536,遗弃罪,"[拒不执行判决、裁定罪, 寻衅滋事罪, 拐卖妇女、儿童罪, 窝藏、包庇罪, 诬告陷害罪]","[拒不执行判决、裁定罪, 非法处置查封、扣押、冻结的财产罪, 寻衅滋事罪, 窝藏、包庇罪, ...","[拒不执行判决、裁定罪, 寻衅滋事罪, 诬告陷害罪, 敲诈勒索罪, 窝藏、包庇罪]"
